<a href="https://colab.research.google.com/github/HLCV-23/Inpainting-Detection/blob/Nicola/automatic_inpainting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install diffusers
!pip install accelerate
!pip install xformers
import numpy as np
from diffusers import StableDiffusionInpaintPipeline
from transformers import pipeline
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd
import time
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.1/109.1 MB 9.0 MB/s eta 0:00:00


In [ ]:
device = torch.device("cuda:0")

mask_generator = pipeline("mask-generation", model="facebook/sam-vit-base", device=device)

#caption_generator = pipeline("image-to-text", model="nlpconnect/vit-gpt2-image-captioning")
caption_generator = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")  # Is BLIP fast enough?

tokenizer = AutoTokenizer.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
model = AutoModelForCausalLM.from_pretrained("Gustavosta/MagicPrompt-Stable-Diffusion")
prompt_generator = pipeline("text-generation", model=model, tokenizer = tokenizer, device = device)

inpainting_generator = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16,
    safety_checker=None,
    low_cpu_mem_usage = False
).to(device)

vae/diffusion_pytorch_model.safetensors not found


Fetching 14 files:   0%|          | 0/14 [00:00<?, ?it/s]

You have disabled the safety checker for <class 'diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion_inpaint.StableDiffusionInpaintPipeline'> by passing `safety_checker=None`. Ensure that you abide to the conditions of the Stable Diffusion license and do not expose unfiltered results in services or applications open to the public. Both the diffusers team and Hugging Face strongly recommend to keep the safety filter enabled in all public facing circumstances, disabling it only for use-cases that involve analyzing network behavior or auditing its results. For more information, please have a look at https://github.com/huggingface/diffusers/pull/254 .


In [ ]:
!pip install datasets
from datasets import load_dataset, Dataset, Image
!huggingface-cli login

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 20.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 11.8 MB/s eta 0:00:00

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tok

In [ ]:
dataset = load_dataset("scene_parse_150", use_auth_token = True, streaming = True, split = "train")  # set streaming to false for scenes!
dataset = iter(dataset.shuffle())

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
"""
!mkdir drive/MyDrive/Imagenet_Dataset/
!touch drive/MyDrive/Imagenet_Dataset/Prompts.csv
!mkdir drive/MyDrive/Imagenet_Dataset/Images
!mkdir drive/MyDrive/Imagenet_Dataset/Images/Modified
!mkdir drive/MyDrive/Imagenet_Dataset/Images/Unmodified
!mkdir drive/MyDrive/Imagenet_Dataset/Masks
!mkdir drive/MyDrive/Imagenet_Dataset/Originals
!pwd
"""

/content


In [ ]:
from PIL import Image, ImageFilter

def generate_datapoint_single_mask(data, id, csv_data, path):
    original_img = data["image"].resize((512,512))
    image_area = 512*512
    masks = mask_generator(original_img)["masks"]

    # Only masks with an area not too large or too small are eligible candidates for inpainting
    valid_masks = [mask for idx, mask in enumerate(masks) if (image_area * 0.1 <= mask.sum() <= image_area * 0.6)]

    if (valid_masks != []):
      choice = np.random.choice(np.arange(len(valid_masks)))
      mask = valid_masks[choice]

      mask_img = Image.fromarray((mask * 255).astype(np.uint8).squeeze())
      mask_img = mask_img.convert("RGB")

      # Generate prompt for inpainting model
      caption = caption_generator(original_img)[0]["generated_text"]
      prompts = prompt_generator(caption)[0]["generated_text"]
      prompts = prompts.replace(". ", ", ")
      prompts = prompts.split(",")
      try:
        prompt = prompts[0] + prompts[1]
      except:
        prompt = prompts[0]

      # inpaint
      inpainted_img = inpainting_generator(prompt=prompt, image=original_img, mask_image=mask_img, height=512, width=512).images[0]

      modified_img = Image.composite(inpainted_img, original_img, mask_img.convert("1")).convert("RGB")

      modified_img.save(path + f"/Images/Modified/{id}.png")
      original_img.save(path + f"/Originals/{id}.png")
      mask_img.save(path + f"/Masks/{id}.png")
      csv_data.at[id, "Prompt"] = prompt
      print(f"Saved processed image {id}.png")

    else:
      print(f"No valid mask could be generated. Skipping image {id}.")


def save_unmodified_image(data, id, path):
  original_img = data["image"].resize((512,512))
  original_img.save(path + f"/Images/Unmodified/{id}.png")

In [ ]:
start_time = time.time()

path = "drive/MyDrive/Imagenet_Dataset3/"

try:
  csv_data = pd.read_csv(path + "/Prompts.csv")
except:
  with open(path + "/Prompts.csv", "w+") as f:
    f.write("idx, Prompt")
  csv_data = pd.read_csv(path + "/Prompts.csv")


# TODO: INIT ID SUCH THAT NO IMAGES ARE OVERWRITTEN!!!

start_id = 2000
num_modified = 2000

for id in range(start_id, start_id + num_modified):
  try:
    generate_datapoint_single_mask(next(dataset), id, csv_data, path)
    print(f"Time passed: {time.time() - start_time}")
  except:
    continue

csv_data.to_csv(path + "/Prompts.csv")

In [ ]:
path = "drive/MyDrive/Scenes_Dataset/"

num_unmodified = len([name for name in os.listdir(path + "/Images/Modified/")])
print(num_unmodified)
#for id in range(1655, 1655 + num_unmodified):
#    save_unmodified_image(next(dataset), id, path)

2946
